<h1>CS152 Assignment 2: The 8-puzzle</h1>

<h1> Sadid Bin Hasan [ CS 152 ] </h>


---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
#Importing Necessary Libraries
import numpy as np

#Class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self, puzzle_state, PC_g = 0, PC_f = 0, parent = None):

        self.PC_g = PC_g    # path cost (PC) for g 
        self.PC_f = PC_f    # A* algorithm gives us f(x)= g(x) + h(x) ; inputting the f(x)
        self.parent = parent  # getting the node parent for solving
        self.puzzle_state = puzzle_state    # we use np_array to describe the puzzle

        self.pruned = False   # considerations made for each puzzle Node (to explore or not to explore)
        self.distance = len(puzzle_state)   

    def __str__(self):
        """
        For nodes in the Puzzle, obtain the (n^2-1) Puzzle_state
        """
        return('\n'.join([' '.join(['{}'.format(position, str(self.distance)) 
                for position in array_line]) for array_line in self.puzzle_state]))
        # using the Python Format specifier, we are taking each entry of our Puzzle
        # and using multiple For loops to represent puzzle digits in our respective grid

    def __lt__(self, another_node):
        """
        Measuring the f(x) value for different nodes during A* search
        """
        return self.PC_f < another_node.PC_f


# ATTRIBUTION AND REFERENCES:
# PYTHON FORMAT SPECIFIER FROM STACKOVERFLOW: CAMPBELL, BRIAN. (2015) PYTHON FORMAT SPECIFIER. RETRIEVED FROM: https://stackoverflow.com/questions/29044940/how-can-you-use-a-variable-name-inside-a-python-format-specifier


In [2]:
# using the starting state from the prompt
checking_start_state = [[7,2,4], [5,0,6], [8,3,1]]

#using the Puzzlenode on our starting state
start_state = PuzzleNode(checking_start_state)

#checking our results
print(start_state)


7 2 4
5 0 6
8 3 1


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [3]:
# Misplaced tiles heuristic

def h1(puzzle_state):
    """
    This function returns the number of misplaced tiles, given the board state
    WE HAVE: 
    - the board state as a list of lists
    - the number of misplaced tiles
    """    
    distance = len(puzzle_state)
    # using numpy library we use the reshape to format the PUZZLE_array without data alteration 
    puzzle_state = np.array(puzzle_state).reshape(distance,distance)    
    # for each of the wrong locations of our digits, we identify them
    wrong_location = [location != line for line, location in enumerate(puzzle_state.flatten())]
    # then we calculate the total number of incorrectly positioned digits
    total = sum(wrong_location) - 1
    return(total)

# Reference: Numpy library of Python: https://numpy.org/doc/stable/reference/generated/numpy.reshape.html

# Manhattan distance heuristic
def h2(puzzle_state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    digits_in_grid = (len(puzzle_state) ** 2)
    distance = len(puzzle_state)
    # using numpy library we use the reshape to format the PUZZLE_array without data alteration    
    puzzle_state = np.array(puzzle_state).reshape(distance, distance)
    # for each of the wrong locations of our digits, we identify them  
    existing_puzzle_state = list(puzzle_state.flatten())  
    # using np_flatten, we are taking the array into 1 Dimension for better interpretation
    final_puzzle_state = list(range(distance**2))
    # re creating the grid for calculating the manhattan distance
        
    calculate_M_D = [abs(coordinate_1 % distance - coordinate_2 % distance) + abs(coordinate_1//distance - coordinate_2//distance) for
                      coordinate_1, coordinate_2 in ((existing_puzzle_state.index(position), final_puzzle_state.index(position)) for
                      position in range(1, digits_in_grid))]
    #calculating the manhattan distance

    total = sum(calculate_M_D)
    # calculating the total manhattan distance for our respective puzzle situation
    return(total)
    #obtaining the manhattan distance.

# Extra heuristic for the extension.  If implemented, modify the function below
    # Not using a third heuristic for now

def h3(puzzle_state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

heuristics = [h1, h2, h3]


# EXTENSION ATTEMPT

From GeeksforGeeks (2021), I learned about the concept of inversions and polarity that can be taken into consideration while determining whether a puzzle is solvable or not. When Inversion number is odd in the beginning of the puzzle, then it will be odd throughout the puzzle, and vice versa for "Even" cases. When we complete our puzzle, our inversion number will become 0. But to solve the puzzle, we need an inversion, which is even.

Rules that I discovered: When we make a movement along a row, the inversion remains the same. But when we make a movement along a column, we can increase or decrease inversion by 2, or it remains the same. (GeeksForGeeks, 2021).


Following these rules, and taking inspiration the GeeksForGeeks resource and the Sliding Puzzle blog, I have constructed my following extension code attempt.


ATTRIBUTION AND REFERENCES:
LEARNED ABOUT PROBLEM SOLVING FOR PUZZLES IN GEEKS FOR GEEKS
HERE: https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/

TOOK FURTHER INSPIRATION FOR THE CODING SOLUTION FROM SLIDING PUZZLE BLOG
HERE: https://bit.ly/3bjwiBj

In [4]:
import numpy as np
#importing necessary libraries

def checking_puzzle_solution(puzzle_state):
    """
    We will be trying to determine the inversion for any 8 Puzzle, and determine whether that
    can be solved.
    
    When we make a movement along a row, the inversion remains the same. But when we make a movement along a column, 
    we can increase or decrease inversion by 2, or it remains the same. (GeeksForGeeks, 2021).
    
    ATTRIBUTION AND REFERENCES:
    LEARNED ABOUT PROBLEM SOLVING FOR PUZZLES IN GEEKS FOR GEEKS
    HERE: https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/

    TOOK FURTHER INSPIRATION FOR THE CODING SOLUTION FROM SLIDING PUZZLE BLOG
    HERE: https://bit.ly/3bjwiBj

    Following these rules, and taking inspiration the GeeksForGeeks resource and the Sliding Puzzle blog, 
    I have constructed my following extension code attempt.
    """
    current_puzz_inversion = 0
    # setting initial inversion to zero
    distance = len(puzzle_state)
    # formatting the puzzle for checking whether we can solve it
    digits_in_grid = (len(puzzle_state) ** 2)
    # using numpy library we use the reshape to format the PUZZLE_array without data alteration
    puzzle_state = np.array(puzzle_state).reshape(distance, distance) 
    #identifying the position of the empty space in the puzzle with numpy library
    empty_space_position = tuple(*np.argwhere(puzzle_state == 0))
    # using np_flatten, we are taking the array into 1 Dimension for better interpretation
    all_the_pz_digits = list(puzzle_state.flatten())  

    # checking all the pieces in the puzzle
    for digit in range(digits_in_grid - 1):
      #checking the other successive pieces in the puzzle
        for checking in range(digit+1, digits_in_grid):
          #comparing the positions of the puzzle and measuring the inversion for the puzzle_state
            if all_the_pz_digits[checking] and all_the_pz_digits[digit] and all_the_pz_digits[digit] > all_the_pz_digits[checking]:
                current_puzz_inversion += 1
                #increasing puzzle inversion number if conditions satisfy

    # Case study Implementations for our puzzle 
    # we are using modulus of 2 to check the even or odd for our values
    if (distance % 2 != 0) and (current_puzz_inversion % 2 == 0):
      # checking divisibility by two for puzzle size
        return(True)
    elif (distance % 2 == 0) and (empty_space_position[0] % 2 != 0) and (current_puzz_inversion % 2 != 0):
      # checking size, empty space position and inversion to determine whether we can solve the puzzle
        return(True)
    elif (distance % 2 == 0) and (empty_space_position[0] % 2 == 0) and (current_puzz_inversion % 2 == 0):
      # checking size, empty space position and inversion to determine whether we can solve the puzzle
        return(True)
    else:
        return(False)

In [5]:
check_prompt_start = [[7,2,4], [5,0,6], [8,3,1]]
checking_puzzle_solution(check_prompt_start)

True

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [6]:
#importing necessary libraries
from queue import PriorityQueue

def solvePuzzle(puzzle_state, heuristic = h1):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """

# checking the entry states of our prompt
    distance = len(puzzle_state)
    formatted_puzzle_end = list(range(distance**2))
    # finding the size and dimesion in which our solution will be in
    if not checking_puzzle_solution(puzzle_state):
      # checking whether or not there is a solution here
        print("WE CAN NOT SOLVE THE GIVEN PUZZLE")
        return(0, 0, 0, None, -2)
    
    try:
       # formatting the entry for convenience of interpretation using the numpy reshape function
        puzzle_state = np.array(puzzle_state).reshape(distance, distance)
    except:
        print("CAN NOT SOLVE: WE CAN NOT FORMAT THE ENTRY YOU SUBMITTED")
      # checking out for invalid entry types
        return(0, 0, 0, None, -1) 
    #checing the size of the puzzle and comparing that to our given puzzle to see whether they match in dimension
    our_formatted_puzzle = puzzle_state.flatten()
    if sorted(our_formatted_puzzle) != formatted_puzzle_end:
      # giving out specific error messages, such that we can identify characteristics of the puzzle_state
        print("CAN NOT SOLVE: THE SIZE OF YOUR ENTRY GRID AND THE ACTUAL GRID SIZE DOES NOT MATCH")
        return(0, 0, 0, None, -1)


# We are going to now search for a way to complete the puzzle
  # for the given point / digit in our puzzle, we input the charactersitics of A* search, where F = G + H
    given_point = PuzzleNode(puzzle_state, PC_g = 0, PC_f = heuristic(puzzle_state))
    move_price = {tuple(given_point.puzzle_state.flatten()): given_point}
    # calculating costs for our given point helps us make the decision.

    # Using the Priority queue, we evaluate our options for the given point
    storing_the_queue = PriorityQueue()
    storing_the_queue.put(given_point)

    # based on the properties of the puzzle game, we identify ways in which a digit can move
    correct_route = [(1,0),(0,1),(-1,0),(0,-1)]
    build_up_calculation = 0
    highest_value_in_stored_queue = 0

    # checking our available options for movement
    while not storing_the_queue.empty():
        
        # evaluating our options with highest cost and starting the evaluation process
        if storing_the_queue.qsize() > highest_value_in_stored_queue:
            highest_value_in_stored_queue = storing_the_queue.qsize()
        # finding out the best route for our existing point in the puzzle
        existing_point = storing_the_queue.get()
        if existing_point.pruned:
            continue
        #checking the number of steps required to complete our solution
        # and evaluating options based on number of steps needed
        if list(existing_point.puzzle_state.flatten()) == formatted_puzzle_end:
            print("After ",build_up_calculation," moves, we have solved the puzzle")
            break
        #empty space in the puzzle, gives us the flexibility to move around
        empty_space = tuple(*np.argwhere(existing_point.puzzle_state == 0))
        # as we take steps towards the solution, we are measuring the number of future steps required
        # if we take the available options to us               
        available_options = [tuple([sum(scan) for scan in zip(empty_space, route)] )
                       for route in correct_route] 
        # among all the possible ways we can solve this problem
        sustainable_route = [scan for scan in available_options if
                       (0 <= scan[0] < distance and 0 <= scan[1] < distance)]
        # we are trying to find the most sustainable solution                
        for best_case in sustainable_route:
            PC_g = existing_point.PC_g + 1    

            # we start following the available options
            observe_case = np.copy(existing_point.puzzle_state)
            empty_digit = empty_space
            # we observe the changes after switching our digits with empty space
            observe_case[empty_digit], observe_case[best_case] = observe_case[best_case], observe_case[empty_digit]        
    
            # after travelling a certain route, we take note of our findings
            if tuple(observe_case.flatten()) in move_price:
                if move_price[tuple(observe_case.flatten())].PC_g > PC_g:
                    move_price[tuple(observe_case.flatten())].pruned = True
                    # then we trace the next best move for solving the puzzle
                else:
                    continue 

            # we evaluate which heuristic works best, based on their prices
            price_for_h = heuristic(observe_case)
            # we find out the available option after observation
            available_op = PuzzleNode(observe_case, PC_g = PC_g,
                                   PC_f = (PC_g + price_for_h),  parent=existing_point)
            # we place our option in the queue
            storing_the_queue.put(available_op)
            # we track the price of our case
            move_price[tuple(observe_case.flatten())] = available_op 
        # finally we keep track of the movements we have made
        build_up_calculation = build_up_calculation + 1

# Now, we do not only aim to solve the puzzle
# We aim to solve the puzzle in the best possible way    
    best_choices_available = []    
    reaching_marker = existing_point
    # trying to find the best way to go ahead

    # comparing the options available to us and placing them among the choices
    while reaching_marker:
      best_choices_available.append(reaching_marker)
      reaching_marker = reaching_marker.parent
      #identifying parent nodes and tracing the best way ahead
    
    best_choice = []
    # after finding the top choices, we now want to make the absolute best choice
    
    # we look at all the options from a different angle
    for iteration, choice in reversed(list(enumerate(best_choices_available))): 
        opt = len(best_choices_available) - 1 - iteration
        best_choice.append(choice.puzzle_state.tolist())
        # and we cross verify, whether our solution is actually the best one or not
    
    #finally, we get the best solution, and we also identify number of total steps needed to solve the puzzle
    return (len(best_choices_available)-1, build_up_calculation, 
            highest_value_in_stored_queue, best_choice, 0)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [7]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

CAN NOT SOLVE: THE SIZE OF YOUR ENTRY GRID AND THE ACTUAL GRID SIZE DOES NOT MATCH


In [8]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [9]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

After  10  moves, we have solved the puzzle
After  9  moves, we have solved the puzzle


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [10]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

WE CAN NOT SOLVE THE GIVEN PUZZLE


In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission